# Load testing set from nodesData.csv and select feature that we need.

In [1]:
path_data='D:\\Career\\DS-ML\\1cademy\\research\\data\\testing\\'
import os
import pandas as pd

#load data
os.chdir(path_data)
df = pd.read_csv('nodesData.csv')
print(df.columns) 

#select the columns we need 
df=df[['id','children','parents','height','title']]
(df)

Index(['id', 'admin', 'changedAt', 'children', 'childrenLength', 'choices',
       'choicesLength', 'closedHeight', 'comments', 'content', 'corrects',
       'createdAt', 'deleted', 'height', 'maxVersionRating', 'nodeImage',
       'nodeType', 'parents', 'parentsLength', 'references',
       'referencesLength', 'studied', 'tags', 'tagsLength', 'title',
       'updatedAt', 'versions', 'viewers', 'wrongs'],
      dtype='object')


,id,children,parents,height,title
0,0059ccSkCvCppWgGPQ9G,"[{""node"":""GtyDY4Dtl1loG0CNy5Lz"",""title"":""Metho...","[{""title"":""Further approaches that could lead ...",210.0,Handling more complicated context\n
1,00NwvYhgES9mjNQ9LRhG,[],"[{""title"":""Drugs working to inactivate transmi...",147.0,Cocaine
2,00bwtgMsbiN0B5HkAxfE,"[{""node"":""DUmRznFNxDg3tz2180xv"",""label"":"""",""ti...","[{""node"":""PDQ3oqH0iMXh74xNdLLn"",""title"":""Emoti...",125.0,Testing Emotional Regulation
3,00mgqGYES5nr2AXUchws,[],"[{""label"":"""",""node"":""wot1mVqtGrFP82NtvnZQ"",""ti...",252.0,Impact of the COVID‐19 pandemic on telehealth ...
4,00yXUI71Epq9q8b9xH9K,"[{""node"":""748r8Mf8cjaxnVh6VKns"",""title"":""Refer...","[{""label"":"""",""node"":""x5zFnnaaioBUd4iU2Y4w"",""ti...",367.0,First Positive Cases in Cats and Dogs in USA
...,...,...,...,...,...
36750,zzS0u9ZJD1hbIG3b7TiH,[],"[{""label"":"""",""title"":""Detection Methods Used C...",220.0,Detection Methods Used Currently by WHO
36751,zzWNrYf8GjnXBqkf9mJQ,"[{""title"":""Methods for determining odds of abn...","[{""title"":""Findings opposed to use of Azithrom...",334.0,Proportion of abnormal ECG findings in hospit...
36752,zzeuylk97oZMOZ4p6tzQ,[],"[{""node"":""qlTVhbjcDPlKuIxXFN1T"",""title"":""Niels...",199.0,NNG Reference Article: Usability Heuristic 8 -...
36753,zzkoyR3Ke1kRaH5Pf9ZH,"[{""title"":""COVID-19 and TB co-infection - 'Fin...","[{""node"":""HzLfRzANJBbX8nvkq1m5"",""title"":""Refer...",200.0,Journal of Infection Papers Related to Seconda...


# Preprocess dataframe to dictionary
{_nodeCo3unt:int,\
_edgeCount: int,\
_nodes:dictionary{nodeKey:(nodeHeight,nodeWidth)}\
_children:dictionary{nodeKey:[childrenKey1, key2...]}\
}

In [2]:
from collections import defaultdict
import numpy as np

node_dic=dict()
edge_dic=defaultdict(list)
titles=set()
num_node=0
num_edge=0

#add valid nodes to node_dic
for i in range(len(df.id)):
    if not np.isnan(df['height'][i]):
        num_node+=1
        node_dic[df['id'][i]]=(600,float(df['height'][i]))
        titles.add(df['title'][i])

#add valid node and children relation to edge dictionary
for i in range(len(df.id)):
    if not np.isnan(df['height'][i]):
        data='g='+df['children'][i]
        data=data.replace('true','True')
        data=data.replace('false','False')
        lcls = locals()
        exec(data, globals(), lcls )
        g = lcls["g"]
        for children in g:
            if children['node'] in node_dic.keys():
                num_edge+=1
                edge_dic[df['id'][i]].append(children['node'])
        
print(num_node)
print(num_edge)


29766
30938


# Load the function that process the output from dagre to DGL graph

In [3]:
import dgl
from dgl.data import DGLDataset
import torch
import itertools

def process_single_graph(data):#input is str directly read from one txt file
    #-----------data cleaning and rearranging--------
    data=data.replace('true','True')
    data=data.replace('false','False')
    data='g='+str(data)
    lcls = locals()
    exec(data, globals(), lcls )
    g = lcls["g"]
    #print(g)
    #------------------------ process to attributes
    G = dgl.DGLGraph()
    G.add_nodes(g['_nodeCount'])
    #print(g['_nodeCount'])
    nodes_ordered=[k for k, v in sorted(nodes.items(), key=lambda item: item[1])]
    in_=[]
    out_=[]
    #print(len(g['_edgeObjs'].keys()))
    for e in g['_edgeObjs'].keys():
        out_.append(nodes[(g['_edgeObjs'][e]['v'][1:])])
        in_.append(nodes[(g['_edgeObjs'][e]['w'][1:])])
    out_=torch.tensor(out_,dtype=torch.long)
    in_=torch.tensor(in_,dtype=torch.long)
    G.add_edges(out_,in_)
    #scale both length and position with output canvas width and height.
    #This way, the trained model assume that the canvas is bounded, but the input size fit just right with canvas size
    width=g['_label']['width']
    height=g['_label']['height']

    #scale both length and position with output canvas width and height.
    #This way, the trained model assume that the canvas is boundless
    width=50
    height=50
    x_=[(g['_nodes']['c'+node]['width']/width,g['_nodes']['c'+node]['height']/height) for node in nodes_ordered]
    y_=[(g['_nodes']['c'+node]['x']/width,g['_nodes']['c'+node]['y']/height) for node in nodes_ordered]
    
    y_=list(itertools.chain.from_iterable(y_))
    #y_=torch.tensor(y_,dtype=torch.double)
    #print(y_)
    G.ndata['x']=torch.tensor(x_,dtype=torch.double)
    try:
        n_edge=g['_edgeCount']
    except:
        n_edge=0
    G.edata['x']=torch.tensor(np.zeros((n_edge,1)),dtype=torch.double)
    return (G,y_)

Using backend: pytorch


# simulate the way user extract subgraphs, sample subgraph, process it using dagre and DGL
1. select a fix number of inital nodes, range from 1 to 10, uniform dist.
2. for each initial nodes, select certain number of children nodes.
3. for the children selected from 2, select certain percentage of children nodes
4. repeat step 3 for K times as long as there'are still children nodes. K is random number from 2 to 10, uniform dist.
5. After finish sampling a subgraph, save it as JSON file, process it using dagre module in javascript and get node positions, reload the dagre-processed graph and format it using DGL.

In [13]:
import random
import json
import os
import subprocess
import pickle
from collections import Counter
from tqdm import tqdm

os.chdir('D:\\Career\\DS-ML\\1cademy\\research\\data\\testing\\')
num_samples=100000
nodes_wl_child=list(edge_dic.keys())
graphs = []
labels = []
times=[]

random.seed(911)#10,22,31，98
for i in tqdm(range(num_samples)):
    #if i%1000==0:print(i)
    num_seed_node=int(np.random.uniform(0,1)*(15+1-7))+7
    K=int(np.random.uniform(0,1)*(10+1-3))+3
    nodes=dict()
    edges=dict()
    dag=dict({'nodes':list(),'edges':list()})
    idx=0
    nodes_cur=random.sample(list(node_dic),num_seed_node)
    G = dgl.DGLGraph()
    for j in range(K):
        for node in nodes_cur.copy():
            #print(j)
            nodes_cur.pop(0)
            if node not in nodes.keys():
                nodes[node]=idx
                dag['nodes'].append({'label':'c'+node,'width':node_dic[node][0],'height':node_dic[node][1]})
                idx+=1
                num_children=len(edge_dic[node])
                if num_children>5:
                    num_children=int(np.random.uniform(0,1)*(5+1-3))+3
                elif num_children==0:
                    continue
                children=random.sample(edge_dic[node],num_children)
                #children=edge_dic[node]
                nodes_cur+=children
                edges[node]=children
                dag['edges']+=[{'parent':'c'+node,'child':'c'+child} for child in children]
    for node in nodes_cur:
        if node not in nodes.keys():
            dag['nodes'].append({'label':'c'+node,'width':float(node_dic[node][0]),'height':float(node_dic[node][1])})
            nodes[node]=idx
            idx+=1
    
    #there is no duplicate in nodes and dag['nodes'], but there could be duplicates in dag['edges'], and it could be 0
    l=[(i['parent'],i['child']) for i in dag['edges']]
    dag['n_nodes']=len(dag['nodes'])
    dag['n_edges']=len(Counter(l))
    with open('data.json', 'w', encoding="utf-8") as f:
        json.dump(dag, f)  
    subprocess.run('node index.js',shell=True, capture_output=True)
    filename='graph.txt'
    text_file = open(filename, "r")
    data = text_file.read()
    text_file.close()
    filename='time.txt'
    text_file = open(filename, "r")
    time = text_file.read()
    text_file.close()
    time=(time.split(','))
    #----------process and add to dataset
    (g,label)=process_single_graph(data)
    graphs.append(g)
    labels.append(label)
    times.append(time)

def trim(y):
    y_len=[len(_) for _ in y]
    y_dim=max(y_len)
    y_dim=500
    yy=([_+[0 for _ in range(y_dim-len(_))] for _ in y])
    return yy
labels=trim(labels)
#remove graphs that have too many nodes
n=len(labels)
for i in range(n-1,-1,-1):
    if len(labels[i])>500:
        graphs.pop(i)
        labels.pop(i)
        times.pop(i)
labels=torch.tensor(labels,dtype=torch.double)


filehandler = open("graphs5.obj","wb")
pickle.dump(graphs,filehandler)
filehandler.close()
filehandler = open("labels5.obj","wb")
pickle.dump(labels,filehandler)
filehandler.close()
filehandler = open("times5.obj","wb")
pickle.dump(times,filehandler)
filehandler.close()


#print(dag['n_nodes'])
#print(dag['n_edges'])

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [15:30:26<00:00,  2.10it/s]


# Codes below are tunning codes (don't run it)

In [17]:
import os
import pickle
import dgl
os.chdir('D:\\Career\\DS-ML\\1cademy\\research\\data\\testing\\')
with open("graphs1.obj", "rb") as f:
    rawdata = f.read()
g1 = pickle.loads(rawdata)
with open("graphs2.obj", "rb") as f:
    rawdata = f.read()
g2 = pickle.loads(rawdata)
with open("graphs3.obj", "rb") as f:
    rawdata = f.read()
g3 = pickle.loads(rawdata)
with open("graphs4.obj", "rb") as f:
    rawdata = f.read()
g4 = pickle.loads(rawdata)
with open("graphs5.obj", "rb") as f:
    rawdata = f.read()
g5 = pickle.loads(rawdata)

g1+=g2+g3+g4+g5
filehandler = open("graphs.obj","wb")
pickle.dump(g1,filehandler)
filehandler.close()

MemoryError: 

In [18]:
print(len(g1))

489723


In [20]:
filehandler = open("graphs.obj","wb")
pickle.dump(g1,filehandler)
filehandler.close()

MemoryError: 

In [211]:
#process_single_graph(data)
#print(dag['edges'])
print(dag['n_edges'])
#process_single_graph(data)
print([(i['child'],i['parent']) in list(Counter(l).keys()) for i in dag['edges']])
#print(list(Counter(l).keys()))
#print(nodes_cur)
#c8WDa0mv5yasgPPPqqW34
np.zeros((0,1))

39
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


array([], shape=(0, 1), dtype=float64)

In [26]:
import dagre_py
import json
import os
import webbrowser
from typing import Dict, List

import pkg_resources

serve_dir = 'D:\\Career\\DS-ML\\1cademy\\research\\data\\testing\\'
json_encoder=None
with open(os.path.join(serve_dir, "data.js"), "w", encoding="utf-8") as fp:
    fp.write("let data = {}".format(json.dumps(dagre_data, cls=json_encoder, indent=2)))
with open('data.json', 'w', encoding="utf-8") as f:
    json.dump(dagre_data, f)
print((json.dumps(dagre_data, cls=json_encoder, indent=2)))

{
  "nodes": [
    {
      "label": "A",
      "attributes": {
        "width": 200,
        "height": 100
      }
    },
    {
      "label": "B",
      "attributes": {
        "width": 300,
        "height": 100
      }
    }
  ],
  "edges": [
    {
      "source": "A",
      "target": "B",
      "label": "edge1"
    }
  ]
}


In [8]:
path_processed="D:\\Career\\DS-ML\\1cademy\\research\\data\\testing\\"

import dgl
from dgl.data import DGLDataset
import torch
import os
import pickle
import numpy as np
import itertools
import zipfile

def process_single_graph(data):#input is str directly read from one txt file
    #-----------data cleaning and rearranging--------
    data=data.replace('true','True')
    data=data.replace('false','False')
    data='g='+str(data)
    lcls = locals()
    exec(data, globals(), lcls )
    g = lcls["g"]
    #------------------------ process to attributes
    G = dgl.DGLGraph()
    G.add_nodes(g['_nodeCount'])
    in_=[]
    out_=[]
    for e in g['_edgeObjs'].keys():
        out_.append(int(g['_edgeObjs'][e]['v'][1:]))
        in_.append(int(g['_edgeObjs'][e]['w'][1:]))
    out_=torch.tensor(out_,dtype=torch.long)
    in_=torch.tensor(in_,dtype=torch.long)
    G.add_edges(out_,in_)
    #scale both length and position with output canvas width and height.
    #This way, the trained model assume that the canvas is bounded, but the input size fit just right with canvas size
    width=g['_label']['width']
    height=g['_label']['height']

    #scale both length and position with output canvas width and height.
    #This way, the trained model assume that the canvas is boundless
    width=1500
    height=1500

    x_=[(g['_nodes']['n'+str(n)]['width']/width,g['_nodes']['n'+str(n)]['height']/height) for n in range(g['_nodeCount'])]
    #for xx in x_:
    #  if int(xx[1])==0:
    #    print(("degenerate box",i,j))
    #    print(x_)
    #    break
    #x_=[(g['_nodes']['n'+str(n)]['width'],g['_nodes']['n'+str(n)]['height']) for n in range(g['_nodeCount'])]
    y_=[(g['_nodes']['n'+str(n)]['x']/width,g['_nodes']['n'+str(n)]['y']/height) for n in range(g['_nodeCount'])]
    #y_=[(g['_nodes']['n'+str(n)]['x'],g['_nodes']['n'+str(n)]['y']) for n in range(g['_nodeCount'])]

    y_=list(itertools.chain.from_iterable(y_))
    #y_=torch.tensor(y_,dtype=torch.double)
    #print(y_)
    G.ndata['x']=torch.tensor(x_,dtype=torch.double)
    G.edata['x']=torch.tensor(np.zeros((g['_edgeCount'],1)),dtype=torch.double)
    return (G,y_)

#--------iterate over all files to construct graph dataset--------
sfix=['']+[' ('+str(i+1)+')' for i in range(n_zip)]#filename suffix
graphs = []
labels = []
times=[]
ct=0
os.chdir(path_data1)
for i in sfix:
    zipname='training'+i+'.zip'
    archive = zipfile.ZipFile(zipname, 'r')
    for j in range(n_inzip):
        ct+=1
        #-----------load data----
        filename='graph'+str(j)+'.txt'
        data = (archive.read(filename).decode("utf-8"))
        filename='time'+str(j)+'.txt'
        time = (archive.read(filename).decode("utf-8"))
        time=(time.split(','))
        #----------process and add to dataset
        (g,label)=process_single_graph(data)
        graphs.append(g)
        labels.append(label)
        times.append(time)
# make sure all sublists have same dimension and convert it torch tensor
# Convert the label list to tensor for saving.
def trim(y):
    y_len=[len(_) for _ in y]
    y_dim=max(y_len)
    y_dim=256
    yy=([_+[0 for _ in range(y_dim-len(_))] for _ in y])
    return yy
labels=trim(labels)
labels=torch.tensor(labels,dtype=torch.double)

os.chdir(path_data2)
filehandler = open("graphs.obj","wb")
pickle.dump(graphs,filehandler)
filehandler.close()
filehandler = open("labels.obj","wb")
pickle.dump(labels,filehandler)
filehandler.close()
filehandler = open("times.obj","wb")
pickle.dump(times,filehandler)
filehandler.close()
print(times)



'[{"node":"GtyDY4Dtl1loG0CNy5Lz","title":"Methods for handling more complicated context in RE","label":""}]'

In [1]:
2**6

64

In [8]:
decay_epoch=25
num_epochs=80
lr_sched_step = lambda epoch: 1 - max(0, epoch - decay_epoch) / (num_epochs - decay_epoch)

for i in range(80):
    (lr_sched_step(i))

In [11]:
import torch
import torch.nn.functional as F

def _discriminator_loss(logits, labels):
    return torch.mean(F.binary_cross_entropy_with_logits(logits, labels, reduction='none'))

pred=torch.tensor([100,200,300])
_discriminator_loss(pred, torch.ones_like(pred))

RuntimeError: result type Float can't be cast to the desired output type Long

In [14]:
import tensorflow as tf
import numpy as np
def int64_feature(value):
    if type(value) != list:
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def float_feature(value):
    if type(value) != list:
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

writer = tf.python_io.TFRecordWriter('file.tfrecords')
bytes = np.array(1.1).tostring() 
int = 1
float = 1.1
example = tf.train.Example(features=tf.train.Features(feature={'1': float_feature(float)}))
writer.write(example.SerializeToString())
writer.close()

for str_rec in tf.python_io.tf_record_iterator('file.tfrecords'):
    example = tf.train.Example()
    example.ParseFromString(str_rec)
    str = (example.features.feature['1'].float_list.value[0])
    print(getsizeof(str))

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


NameError: name 'getsizeof' is not defined